<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/llm_schulung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wichtig**: Wenn Du dieses Notebook in Google Colab ausführst, wähle zuerst unter "Laufzeit > Laufzeittyp ändern" eine GPU aus, um die Ausführung zu beschleunigen.

# Transformer nutzen

Zunächst installieren wir die benötigte Library Huggingface Transformer

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00


## Klassifizierung von Text

Wir laden eine Pipeline mit einem vortrainierten Sentiment-Modell. Das Modell kann explizit angegeben werden, ansonsten wird ein Standard-Modell (`distilbert-base-uncased-finetuned-sst-2-english`) genommen.

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Wir können das Modell nutzen, um die Stimmung eines Textes zu bestimmen. Hierbei wird angegeben, ob der Text positiv oder negativ ist und wie sicher das Modell ist bei der Bewertung.

In [3]:
classifier(["We are happy to have you participate in our modest LLM workshop!", "Sorry that we had to cap the participant limit to 20"])

[{'label': 'POSITIVE', 'score': 0.9998558759689331},
 {'label': 'NEGATIVE', 'score': 0.9991243481636047}]

Dieses Modell wurde nur mit englischen Texten und scheinbar nicht mit Emoji trainiert:

In [4]:
classifier(["Was ein schöner Tag für eine Schulung", "🤗", "😠"])

[{'label': 'NEGATIVE', 'score': 0.9137821793556213},
 {'label': 'NEGATIVE', 'score': 0.6970565319061279},
 {'label': 'NEGATIVE', 'score': 0.6970565319061279}]

## Textgenerierung

Wir laden das vortrainierte distilgpt2 Modell und nutzen es für Textgenerierung

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

Wir können das Modell nutzen, um Texte zu vervollständigen.

In [6]:
from transformers import set_seed
set_seed(42)
pipe("Hello, I’m a language model", max_length=20, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I’m a language model but what I do in that role is to make everything'},
 {'generated_text': 'Hello, I’m a language model.” You’ll know the real name'},
 {'generated_text': 'Hello, I’m a language model’s\xadself, and I’m'},
 {'generated_text': 'Hello, I’m a language model, and I really wanted to make a nice API,'},
 {'generated_text': 'Hello, I’m a language model. I’think in any language. I�'}]

## Aufgabe: Nutzt Transformer auf andere Weise!

Spielt ein wenig mit der Pipeline rum:
- Probiert andere Texte aus
- Vergleicht verschiedene Modelle
- Nutzt einen anderen Anwendungsfall

Auf folgender Seite findet ihr weitere mögliche Anwendungsfälle
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Probiert gerne was aus 🤗

# Transformer verstehen

Nun schauen wir in die Pipeline rein, um zu verstehen, wie Transformer funktionieren.

Die Pipeline stellt unser gesamtes Language Model dar. Sie besteht bei Transformern aus einem Tokenizer und dem eigentlichen Neuronalen Netz.

In [7]:
tokenizer = pipe.tokenizer
model = pipe.model

## Tokenizer

Der Tokenizer in einem (Large) Language Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [8]:
text = "Hello, I’m a language model"
tokenizer.tokenize(text)

['Hello', ',', 'ĠI', 'âĢ', 'Ļ', 'm', 'Ġa', 'Ġlanguage', 'Ġmodel']

In [9]:
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[15496,    11,   314,   447,   247,    76,   257,  3303,  2746]])

Der Tokenizer enthält ein Dictionary, welches jedem Token eine Zahl zuweist

In [10]:
vocab = tokenizer.get_vocab()
first_20_items = {k: vocab[k] for k in list(vocab.keys())[:20]}
first_20_items

{'Ġweld': 26650,
 'Ġlyrics': 15844,
 'Ġaisle': 28960,
 'ĠDanny': 15105,
 'Ġreductions': 20691,
 'bro': 7957,
 'Ġhob': 32724,
 'ĠBurgess': 46744,
 'fl': 2704,
 'Islamic': 26723,
 'ENDED': 49361,
 'ĠStudent': 13613,
 'Ġdeclare': 13627,
 'Ġfirewall': 32928,
 'ĠHimself': 31657,
 'Ġunsuccessfully': 40665,
 'ĠHam': 4345,
 'Ġcaste': 30747,
 'ĠSicily': 49301,
 'Ġconsultancy': 47827}

Es gibt insgesamt 50257 Tokens in diesem Tokenizer.

In [11]:
len(vocab)

50257

## Transformer-Modell

Das Modell ist der eigentliche Transformer, der den encodierten Text entgegen nimmt und verarbeitet oder *transformiert* 😎.

### Die Ausgabe verstehen

Statt die Pipeline als ganzes zu nutzen, können wir auch manuell die Ausgabe des Encoders durch unser Modell schleusen, um zu sehen, was es damit macht.

Obwohl wir nur den fehlenden Text vervollständigen wollen, liefert unser Transformer tatsächlich für **jedes** Token in unserem Input-Text eine Vorhersage für das nächste Token:

In [12]:
output = model(**encoded_input)
token_logits = output.logits

# Get the predicted token ids
predicted_token_ids = token_logits.argmax(dim=-1)
predicted_token_ids

tensor([[ 383,  290,  447,  247,   76,  407, 4336, 5887,   13]])

In [13]:
# Decode the tokens to text
predicted_text = [tokenizer.decode(t) for t in predicted_token_ids]

for index, t in enumerate(predicted_token_ids[0]):
  input_token = tokenizer.tokenize(text)[index]
  predicted_next_token = tokenizer.decode(t)
  print(input_token + " => " + predicted_next_token)


Hello =>  The
, =>  and
ĠI => �
âĢ => �
Ļ => m
m =>  not
Ġa =>  fan
Ġlanguage =>  expert
Ġmodel => .


Das liegt daran, dass Transformer auf massiv parallele Verarbeitung und paralleles Training ausgelegt sind. Deswegen verarbeiten sie immer den gesamten Text parallel.

Um eine größere Spanne von Texten erzeugen zu können, nutzen Transformer nicht immer die wahrscheinlichste Vorhersage (Stichwort **Temperatur**).

Hier sind die fünf wahrscheinlichsten Vorhersagen:

In [14]:
import torch

# Convert logits to probabilities
token_probs = torch.nn.functional.softmax(token_logits, dim=-1)

# Get the top 5 predicted token ids
top_k = 5
top_k_probs, top_k_indices = token_probs.topk(top_k, dim=-1)

# Iterate over the tokens and print the top 5 predictions for each one
for i in range(top_k_indices.shape[1]):
    input_token = tokenizer.tokenize(text)[i]
    print(f"Input: {input_token}:")
    for j in range(top_k):
        token_id = top_k_indices[0, i, j].item()
        probability = top_k_probs[0, i, j].item()
        output_token = tokenizer.decode([token_id])
        print(f"    Output: {output_token}, probability: {probability:.2%}")


Input: Hello:
    Output:  The, probability: 4.25%
    Output: ., probability: 1.89%
    Output: ,, probability: 1.78%
    Output:  A, probability: 1.26%
    Output: The, probability: 1.19%
Input: ,:
    Output:  and, probability: 8.29%
    Output:  the, probability: 4.59%
    Output:  we, probability: 4.25%
    Output:  I, probability: 3.78%
    Output:  you, probability: 3.22%
Input: ĠI:
    Output: �, probability: 16.13%
    Output:  am, probability: 7.07%
    Output: 'm, probability: 6.29%
    Output:  have, probability: 4.64%
    Output:  think, probability: 4.03%
Input: âĢ:
    Output: �, probability: 39.79%
    Output: �, probability: 11.00%
    Output: �, probability: 7.78%
    Output: �, probability: 4.51%
    Output: �, probability: 4.34%
Input: Ļ:
    Output: m, probability: 53.07%
    Output: ve, probability: 13.96%
    Output: ll, probability: 8.22%
    Output: d, probability: 3.58%
    Output: �, probability: 1.60%
Input: m:
    Output:  not, probability: 10.25%
    Outpu

### Die Architektur verstehen


In [14]:
!pip install transformers einops accelerate

In [15]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Laden des vortrainierten Modells und des zugehörigen Tokenizers
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)

# GPU nutzen wenn verfügar
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device).eval()

text = 'Das ist ein Beispieltext für die Demonstration.'

tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Verfolgen der Veränderungen in den latenten Zuständen und Bestätigen der konstanten Dimension.
# Dazu führen wir die Eingabe durch das gesamte Modell und drucken die Größe der Ausgabe nach jedem Block
input_tensor = torch.tensor([input_ids], dtype=torch.long).unsqueeze(0).to(device)

with torch.no_grad():
    # Durchführen einer Vorwärtsdurchlaufe durch das Modell
    outputs = model(input_tensor)
    # Die Ausgabe ist ein Tuple, das die Endausgabe und eine Liste der Zwischenausgaben enthält
    final_output, hidden_states = outputs.last_hidden_state, outputs.hidden_states
    # Drucken Sie die Größe der Endausgabe und jeder Zwischenausgabe
    print(f"Größe der Endausgabe:\n\t{final_output.size()}\n")
    print(f"Endausgabe:\n\t{final_output}\n")
    for i, hidden_state in enumerate(hidden_states):
        print(f"Größe der latenten Zustände nach Block {i+1}:\n\t{hidden_state.size()}\n")
        print(f"Latente Zustände nach Block {i+1}:\n\t{hidden_state}\n\n")

Größe der Endausgabe:
	torch.Size([1, 1, 16, 768])

Endausgabe:
	tensor([[[[-0.1805, -0.1602, -0.2445,  ..., -0.1593,  0.0039, -0.0136],
          [-0.3021, -0.3064, -0.4753,  ...,  0.0716, -0.1752,  0.2778],
          [-0.6312, -0.3724, -0.7612,  ...,  0.3610, -0.1038,  0.0860],
          ...,
          [ 0.6071, -0.1206, -0.9526,  ..., -0.4354,  0.0310, -0.5339],
          [ 0.0030, -0.4362, -0.2058,  ..., -0.1486, -0.3879, -0.3260],
          [ 0.0698, -0.4597, -0.5089,  ..., -0.2768, -0.2152, -0.0472]]]],
       device='cuda:0')

Größe der latenten Zustände nach Block 1:
	torch.Size([1, 16, 768])

Latente Zustände nach Block 1:
	tensor([[[ 0.0911, -0.1488,  0.1890,  ..., -0.0553,  0.1373, -0.0367],
         [-0.0502, -0.2643, -0.0026,  ...,  0.1705, -0.0574, -0.0334],
         [-0.0055, -0.0747,  0.1101,  ...,  0.1342, -0.0187, -0.0468],
         ...,
         [-0.0501,  0.0583,  0.4026,  ..., -0.0276, -0.2199, -0.0523],
         [-0.0544,  0.0188,  0.1011,  ..., -0.0354, -0.1995, 